In [5]:
import src.net as net
import sklearn.metrics
import torch.nn as nn
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import importlib
import src.params as params
import src.main as main
import src.dataloader as dataloader
import torch.nn.functional as F
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import StandardScaler
import src.preprocess as preprocess
from copy import copy
import src.feature_extraction as feature_extraction
import sys, os, torch
import re
importlib.reload(dataloader)
importlib.reload(preprocess)
importlib.reload(feature_extraction)
importlib.reload(net)

<module 'src.net' from '/home/junetheriver/code/aiops/src/net.py'>

In [16]:
args = params.get_args()

In [38]:
gt = dataloader.load_gt(args)

In [39]:
gt = preprocess.preprocess_gt(gt)

In [43]:
gt.time.min(), gt.time.max()

(1614281582.259792, 1616634061.622639)

In [11]:
from pandas import DataFrame
import os
import pandas
dt_path = "../../data/system-a"
df = DataFrame()
for file in os.listdir(dt_path)[:1]:
    cmdb_id, kpi_name = os.path.splitext(file)[0].split('##')
    a = pandas.read_csv(os.path.join(dt_path, file), index_col=0)
    print(a.describe())

          timestamp         value
count  2.016000e+04  20160.000000
mean   1.615466e+09      0.042151
std    7.252418e+05      0.924960
min    1.614269e+09      0.000000
25%    1.614571e+09      0.000000
50%    1.615651e+09      0.000000
75%    1.615954e+09      0.000000
max    1.616602e+09     55.660000


In [19]:
importlib.reload(dataloader)
index = dataloader.get_multiIndex(args)

35
216
Loaded with 3261 cmdb_kpi tuples


In [18]:
args = params.get_args("../config/system_a.yaml")

In [10]:
index

MultiIndex([(  'gjjcore8',   'system.net.tcp.retrans_segs'),
            (  'gjjcore9',        'system.tcp.retrans_pct'),
            ( 'gjjredis1',                 'redis.cpu.sys'),
            (  'gjjcore7',             'system.mem.shared'),
            (    'gjjha3',               'system.run_time'),
            (    'gjjha1',                 'system.io.w_s'),
            (   'gjjbps1',                 'system.load.1'),
            (  'gjjcore7',               'system.run_time'),
            (  'gjjcore5',         'system.fs.inodes.free'),
            (  'gjjcore9',        'system.udp.connect.num'),
            ...
            ('gjjvincio2',         'system.fs.inodes.free'),
            (    'gjjha3',               'system.mem.free'),
            (    'gjjha3',             'system.swap.total'),
            (  'gjjcore6',           'system.tcp.last_ack'),
            (  'gjjuaap1',          'system.swap.used_pct'),
            (  'gjjcore6',          'system.tcp.fin_wait1'),
        

In [61]:
df = DataFrame(index=index, columns=["timestamp", "value"])

In [56]:
df.index.values

array([('gjjcore8', 'system.net.tcp.retrans_segs'),
       ('gjjcore9', 'system.tcp.retrans_pct'),
       ('gjjredis1', 'redis.cpu.sys'), ...,
       ('gjjvincio2', 'system.load.norm.5'),
       ('gjjuaap1', 'weblogic.can_connect'),
       ('gjjcore9', 'weblogic.jdbc.active_conn_avg')], dtype=object)

In [47]:
importlib.reload(dataloader)

<module 'src.dataloader' from '/home/junetheriver/code/aiops/src/dataloader.py'>

In [48]:
index[0]

('gjjcore8', 'system.net.tcp.retrans_segs')

In [53]:
df1 = dataloader.get_cmdb_kpi(args, *index[0]).drop(columns=['cmdb_id', 'kpi_name'])

In [63]:
df1.timestamp

313        1614268800
4479       1614268860
7920       1614268920
8636       1614268980
11357      1614269040
              ...    
4021286    1616601300
4022661    1616601360
4024995    1616601420
4028640    1616601480
4032063    1616601540
Name: timestamp, Length: 20160, dtype: int64

In [65]:
(1616601540 - 1614268800) / 60

38879.0

In [66]:
index

MultiIndex([(  'gjjcore8',   'system.net.tcp.retrans_segs'),
            (  'gjjcore9',        'system.tcp.retrans_pct'),
            ( 'gjjredis1',                 'redis.cpu.sys'),
            (  'gjjcore7',             'system.mem.shared'),
            (    'gjjha3',               'system.run_time'),
            (    'gjjha1',                 'system.io.w_s'),
            (   'gjjbps1',                 'system.load.1'),
            (  'gjjcore7',               'system.run_time'),
            (  'gjjcore5',         'system.fs.inodes.free'),
            (  'gjjcore9',        'system.udp.connect.num'),
            ...
            ('gjjvincio2',         'system.fs.inodes.free'),
            (    'gjjha3',               'system.mem.free'),
            (    'gjjha3',             'system.swap.total'),
            (  'gjjcore6',           'system.tcp.last_ack'),
            (  'gjjuaap1',          'system.swap.used_pct'),
            (  'gjjcore6',          'system.tcp.fin_wait1'),
        

In [67]:
len(index)

3261